# 1. Bounded & Unbounded  Wildcards in Generics
----

Type can be upper bounded by using `<? extends T>` where all Types must be sub-class of T or lower bounded using `<? super T>` where all Types required to be the super class of T, here T represent the lower bound. 

`Single <?>` is called an unbounded wildcard in generic and it can represent any types that are similar to `Object` in Java. For example, `List<?>`(actually this will be replaced with `List<Object>` by complier) can represent any List e.g. `List<String>` or `List<Integer>` it provides the highest level of flexibility on passing method argument.

https://javarevisited.blogspot.com/2012/04/what-is-bounded-and-unbounded-wildcards.html

# 2. What is the benefits of Using Generics?
---

看看[java documentation](https://docs.oracle.com/javase/tutorial/java/generics/why.html)解释用generics的目的:

> In a nutshell, generics enable types (classes and interfaces) to be parameters when defining classes, interfaces and methods. Much like the more familiar formal parameters used in method declarations, type parameters **provide a way for you to re-use the same code with different inputs**. The difference is that the inputs to formal parameters are values, while the inputs to type parameters are types. 

### 2.1.  Imporve Type Safety of Java

先看看什么是type safety: 

> Type safety in a programming language is an abstract construct that enables the language to avoid type errors. https://www.baeldung.com/cs/type-safety-programming

A Java compiler applies strong type checking to generic code and issues errors if the code violates type safety. Fixing compile-time errors is easier than fixing runtime errors, which can be difficult to find. 

### 2.2. Elimination of casts
没有generics的时候:
```java
List list = new ArrayList();
list.add("hello");
String s = (String) list.get(0);
```

有了generics我们就不用cast了, 因为有时候会忘记cast, 那就可能会遇到ClassCastException, 所以有了generics之后编译器就可以在编译期帮我们检查容器里元素的类型, 从而减少了ClassCastException的发生, 在一定程度上增加了type safety. 
```java
List<String> list = new ArrayList<String>();
list.add("hello");
String s = list.get(0);   // no cast
```

[再来个例子](https://javarevisited.blogspot.com/2011/09/generics-java-example-tutorial.html): 
For example, HashSet of String will only contain String object and if you try to put Integer or any other object, the compiler will complain. **Before Java 5 same code will pass** compile-time check but will fail at runtime which is worse. Generics allows Java programmers to write more robust and type-safe code.

参考:https://docs.oracle.com/javase/tutorial/java/generics/why.html


# 3. Type Erasure
---

> Java generics only exit during compile time.

> Java的泛型基本上都是在编译器这个层次上实现的，在生成的bytecode中是不包含泛型中的类型信息的，使用泛型的时候加上类型参数，在编译器编译的时候会discard，这个过程成为type erasure 


### 3.1. 通过两个例子证明Java类型的类型擦除

- **原始类型相等**

```java
public class Test {

    public static void main(String[] args) {

        ArrayList<String> list1 = new ArrayList<String>();
        list1.add("abc");

        ArrayList<Integer> list2 = new ArrayList<Integer>();
        list2.add(123);

        System.out.println(list1.getClass() == list2.getClass());
    }

}
```

- **通过反射添加其它类型元素**

```java
public class Test {

    public static void main(String[] args) throws Exception {

        ArrayList<Integer> list = new ArrayList<Integer>();

        list.add(1);  //这样调用 add 方法只能存储整形，因为泛型类型的实例为 Integer

        list.getClass().getMethod("add", Object.class).invoke(list, "asd");

        for (int i = 0; i < list.size(); i++) {
            System.out.println(list.get(i));
        }
    }

}
```

### 3.2. 类型擦除后保留的原始类型
在上面，两次提到了原始类型，什么是原始类型？

原始类型 就是擦除去了泛型信息，最后在字节码中的类型变量的真正类型，无论何时定义一个泛型，相应的原始类型都会被自动提供，类型变量擦除，并使用其bound type（unbound type的变量用Object）替换。

```java
public class Pair<T> {  
    private T value;  
    public T getValue() {  
        return value;  
    }  
    public void setValue(T  value) {  
        this.value = value;  
    }  
}  

// Pair的原始类型为

public class Pair {  
    private Object value;  
    public Object getValue() {  
        return value;  
    }  
    public void setValue(Object  value) {  
        this.value = value;  
    }  
}
```
因为在`Pair<T>`中，`T` 是一个无限定的类型变量，所以用`Object`替换，其结果就是一个普通的类，但如果类型变量是bound，那么原始类型就用第一个边界的类型变量类替换。

比如: Pair这样声明的话, 那么原始类型就是`Comparable` 

```java
public class Pair<T extends Comparable> {}
```

主要参考: https://www.cnblogs.com/wuqinglong/p/9456193.html

# 4. 类型擦除引起的问题及解决方法
---

### 4.1. 自动类型转换

因为类型擦除的问题，所以所有的泛型类型变量最后都会被替换为原始类型。

既然都被替换为原始类型，那么为什么我们在获取`get()`的时候，不需要进行强制类型转换呢？因为`get()`已经帮我们cast了. 

看下`ArrayList.get()`方法：

```java
public E get(int index) {
    Objects.checkIndex(index, size);
    return elementData(index);
}

E elementData(int index) {
    return (E) elementData[index];
}
```

### 4.2. 泛型类型变量不能是基本数据类型
```
ArrayList<int> arr; //error
ArrayList<Integer> arr;
```

不能用类型参数替换基本类型。就比如，没有`ArrayList<double>`，只有`ArrayList<Double>`。因为当类型擦除后，`ArrayList`的原始类型变为`Object`，但是`Object`类型不能存储`double`值，只能引用`Double`的值。

### 泛型在静态方法和静态类中的问题

```java
public class Test2<T> {    
    public static T one;   //编译错误    
    public static  T show(T one){ //编译错误    
        return null;    
    }    
}
```
因为泛型类中的泛型参数的实例化是在定义对象的时候指定的，而静态变量和静态方法不需要使用对象来调用。对象都没有创建，如何确定这个泛型参数是何种类型，所以当然是错误的。


但是要注意区分下面的一种情况：
```java
public class Test2<T> {    

    public static <T >T show(T one){ //这是正确的    
        return null;    
    }    
}
```

因为这是一个泛型方法，在泛型方法中使用的T是自己在方法中定义的 T，而不是泛型类中的T。